In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt, seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

from stop_words import get_stop_words

import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import re

In [ ]:
tweets = pd.read_csv('/kaggle/input/bitcoin-tweets-16m-tweets-with-sentiment-tagged/mbsa.csv')
tweets.head()

In [ ]:
tweets = tweets.iloc[:,1:]
tweets.head()

In [ ]:
tweets.Sentiment = tweets.Sentiment.apply(lambda x: str(x).lower())
tweets.Sentiment = tweets.Sentiment.map(lambda x: 1 if x=='positive' else 0)
tweets.head()

In [ ]:
len(tweets)

In [ ]:
# A sample of datapoints are selected
df = tweets.sample(50000)
len(df)

In [ ]:
df.head()

In [ ]:
languages = ['french','spanish','russian','portuguese']
stop_words = get_stop_words('english')
for i in languages:
    stop_words = stop_words + get_stop_words(i)
    
len(stop_words)

In [ ]:
# Stop words from different languages are added
# Source : Github
jap = '''あそこ
あっ
あの
あのかた
あの人
あり
あります
ある
あれ
い
いう
います
いる
う
うち
え
お
および
おり
おります
か
かつて
から
が
き
ここ
こちら
こと
この
これ
これら
さ
さらに
し
しかし
する
ず
せ
せる
そこ
そして
その
その他
その後
それ
それぞれ
それで
た
ただし
たち
ため
たり
だ
だっ
だれ
つ
て
で
でき
できる
です
では
でも
と
という
といった
とき
ところ
として
とともに
とも
と共に
どこ
どの
な
ない
なお
なかっ
ながら
なく
なっ
など
なに
なら
なり
なる
なん
に
において
における
について
にて
によって
により
による
に対して
に対する
に関する
の
ので
のみ
は
ば
へ
ほか
ほとんど
ほど
ます
また
または
まで
も
もの
ものの
や
よう
より
ら
られ
られる
れ
れる
を
ん
何
及び
彼
彼女
我々
特に
私
私達
貴方
貴方方'''


korean = '''…
、
。
〈
〉
《
》
가
가까스로
가령
각
각각
각자
각종
갖고말하자면
같다
같이
개의치않고
거니와
거바
거의
것
것과 같이
것들
게다가
게우다
겨우
견지에서
결과에 이르다
결국
결론을 낼 수 있다
겸사겸사
고려하면
고로
곧
공동으로
과
과연
관계가 있다
관계없이
관련이 있다
관하여
관한
관해서는
구
구체적으로
구토하다
그
그들
그때
그래
그래도
그래서
그러나
그러니
그러니까
그러면
그러므로
그러한즉
그런 까닭에
그런데
그런즉
그럼
그럼에도 불구하고
그렇게 함으로써
그렇지
그렇지 않다면
그렇지 않으면
그렇지만
그렇지않으면
그리고
그리하여
그만이다
그에 따르는
그위에
그저
그중에서
그치지 않다
근거로
근거하여
기대여
기점으로
기준으로
기타
까닭으로
까악
까지
까지 미치다
까지도
꽈당
끙끙
끼익
나
나머지는
남들
남짓
너
너희
너희들
네
넷
년
논하지 않다
놀라다
누가 알겠는가
누구
다른
다른 방면으로
다만
다섯
다소
다수
다시 말하자면
다시말하면
다음
다음에
다음으로
단지
답다
당신
당장
대로 하다
대하면
대하여
대해 말하자면
대해서
댕그
더구나
더군다나
더라도
더불어
더욱더
더욱이는
도달하다
도착하다
동시에
동안
된바에야
된이상
두번째로
둘
둥둥
뒤따라
뒤이어
든간에
들
등
등등
딩동
따라
따라서
따위
따지지 않다
딱
때
때가 되어
때문에
또
또한
뚝뚝
라 해도
령
로
로 인하여
로부터
로써
륙
를
마음대로
마저
마저도
마치
막론하고
만 못하다
만약
만약에
만은 아니다
만이 아니다
만일
만큼
말하자면
말할것도 없고
매
매번
메쓰겁다
몇
모
모두
무렵
무릎쓰고
무슨
무엇
무엇때문에
물론
및
바꾸어말하면
바꾸어말하자면
바꾸어서 말하면
바꾸어서 한다면
바꿔 말하면
바로
바와같이
밖에 안된다
반대로
반대로 말하자면
반드시
버금
보는데서
보다더
보드득
본대로
봐
봐라
부류의 사람들
부터
불구하고
불문하고
붕붕
비걱거리다
비교적
비길수 없다
비로소
비록
비슷하다
비추어 보아
비하면
뿐만 아니라
뿐만아니라
뿐이다
삐걱
삐걱거리다
사
삼
상대적으로 말하자면
생각한대로
설령
설마
설사
셋
소생
소인
솨
쉿
습니까
습니다
시각
시간
시작하여
시초에
시키다
실로
심지어
아
아니
아니나다를가
아니라면
아니면
아니었다면
아래윗
아무거나
아무도
아야
아울러
아이
아이고
아이구
아이야
아이쿠
아하
아홉
안 그러면
않기 위하여
않기 위해서
알 수 있다
알았어
앗
앞에서
앞의것
야
약간
양자
어
어기여차
어느
어느 년도
어느것
어느곳
어느때
어느쪽
어느해
어디
어때
어떠한
어떤
어떤것
어떤것들
어떻게
어떻해
어이
어째서
어쨋든
어쩔수 없다
어찌
어찌됏든
어찌됏어
어찌하든지
어찌하여
언제
언젠가
얼마
얼마 안 되는 것
얼마간
얼마나
얼마든지
얼마만큼
얼마큼
엉엉
에
에 가서
에 달려 있다
에 대해
에 있다
에 한하다
에게
에서
여
여기
여덟
여러분
여보시오
여부
여섯
여전히
여차
연관되다
연이서
영
영차
옆사람
예
예를 들면
예를 들자면
예컨대
예하면
오
오로지
오르다
오자마자
오직
오호
오히려
와
와 같은 사람들
와르르
와아
왜
왜냐하면
외에도
요만큼
요만한 것
요만한걸
요컨대
우르르
우리
우리들
우선
우에 종합한것과같이
운운
월
위에서 서술한바와같이
위하여
위해서
윙윙
육
으로
으로 인하여
으로서
으로써
을
응
응당
의
의거하여
의지하여
의해
의해되다
의해서
이
이 되다
이 때문에
이 밖에
이 외에
이 정도의
이것
이곳
이때
이라면
이래
이러이러하다
이러한
이런
이럴정도로
이렇게 많은 것
이렇게되면
이렇게말하자면
이렇구나
이로 인하여
이르기까지
이리하여
이만큼
이번
이봐
이상
이어서
이었다
이와 같다
이와 같은
이와 반대로
이와같다면
이외에도
이용하여
이유만으로
이젠
이지만
이쪽
이천구
이천육
이천칠
이천팔
인 듯하다
인젠
일
일것이다
일곱
일단
일때
일반적으로
일지라도
임에 틀림없다
입각하여
입장에서
잇따라
있다
자
자기
자기집
자마자
자신
잠깐
잠시
저
저것
저것만큼
저기
저쪽
저희
전부
전자
전후
점에서 보아
정도에 이르다
제
제각기
제외하고
조금
조차
조차도
졸졸
좀
좋아
좍좍
주룩주룩
주저하지 않고
줄은 몰랏다
줄은모른다
중에서
중의하나
즈음하여
즉
즉시
지든지
지만
지말고
진짜로
쪽으로
차라리
참
참나
첫번째로
쳇
총적으로
총적으로 말하면
총적으로 보면
칠
콸콸
쾅쾅
쿵
타다
타인
탕탕
토하다
통하여
툭
퉤
틈타
팍
팔
퍽
펄렁
하
하게될것이다
하게하다
하겠는가
하고 있다
하고있었다
하곤하였다
하구나
하기 때문에
하기 위하여
하기는한데
하기만 하면
하기보다는
하기에
하나
하느니
하는 김에
하는 편이 낫다
하는것도
하는것만 못하다
하는것이 낫다
하는바
하더라도
하도다
하도록시키다
하도록하다
하든지
하려고하다
하마터면
하면 할수록
하면된다
하면서
하물며
하여금
하여야
하자마자
하지 않는다면
하지 않도록
하지마
하지마라
하지만
하하
한 까닭에
한 이유는
한 후
한다면
한다면 몰라도
한데
한마디
한적이있다
한켠으로는
한항목
할 따름이다
할 생각이다
할 줄 안다
할 지경이다
할 힘이 있다
할때
할만하다
할망정
할뿐
할수있다
할수있어
할줄알다
할지라도
할지언정
함께
해도된다
해도좋다
해봐요
해서는 안된다
해야한다
해요
했어요
향하다
향하여
향해서
허
허걱
허허
헉
헉헉
헐떡헐떡
형식으로 쓰여
혹시
혹은
혼자
훨씬
휘익
휴
흐흐
흥
힘입어'''


chinese = '''一
一个
一些
一何
一切
一则
一方面
一旦
一来
一样
一种
一般
一转眼
七
万一
三
上
上下
下
不
不仅
不但
不光
不单
不只
不外乎
不如
不妨
不尽
不尽然
不得
不怕
不惟
不成
不拘
不料
不是
不比
不然
不特
不独
不管
不至于
不若
不论
不过
不问
与
与其
与其说
与否
与此同时
且
且不说
且说
两者
个
个别
中
临
为
为了
为什么
为何
为止
为此
为着
乃
乃至
乃至于
么
之
之一
之所以
之类
乌乎
乎
乘
九
也
也好
也罢
了
二
二来
于
于是
于是乎
云云
云尔
五
些
亦
人
人们
人家
什
什么
什么样
今
介于
仍
仍旧
从
从此
从而
他
他人
他们
他们们
以
以上
以为
以便
以免
以及
以故
以期
以来
以至
以至于
以致
们
任
任何
任凭
会
似的
但
但凡
但是
何
何以
何况
何处
何时
余外
作为
你
你们
使
使得
例如
依
依据
依照
便于
俺
俺们
倘
倘使
倘或
倘然
倘若
借
借傥然
假使
假如
假若
做
像
儿
先不先
光
光是
全体
全部
八
六
兮
共
关于
关于具体地说
其
其一
其中
其二
其他
其余
其它
其次
具体地说
具体说来
兼之
内
再
再其次
再则
再有
再者
再者说
再说
冒
冲
况且
几
几时
凡
凡是
凭
凭借
出于
出来
分
分别
则
则甚
别
别人
别处
别是
别的
别管
别说
到
前后
前此
前者
加之
加以
区
即
即令
即使
即便
即如
即或
即若
却
去
又
又及
及
及其
及至
反之
反而
反过来
反过来说
受到
另
另一方面
另外
另悉
只
只当
只怕
只是
只有
只消
只要
只限
叫
叮咚
可
可以
可是
可见
各
各个
各位
各种
各自
同
同时
后
后者
向
向使
向着
吓
吗
否则
吧
吧哒
含
吱
呀
呃
呕
呗
呜
呜呼
呢
呵
呵呵
呸
呼哧
咋
和
咚
咦
咧
咱
咱们
咳
哇
哈
哈哈
哉
哎
哎呀
哎哟
哗
哟
哦
哩
哪
哪个
哪些
哪儿
哪天
哪年
哪怕
哪样
哪边
哪里
哼
哼唷
唉
唯有
啊
啐
啥
啦
啪达
啷当
喂
喏
喔唷
喽
嗡
嗡嗡
嗬
嗯
嗳
嘎
嘎登
嘘
嘛
嘻
嘿
嘿嘿
四
因
因为
因了
因此
因着
因而
固然
在
在下
在于
地
基于
处在
多
多么
多少
大
大家
她
她们
好
如
如上
如上所述
如下
如何
如其
如同
如是
如果
如此
如若
始而
孰料
孰知
宁
宁可
宁愿
宁肯
它
它们
对
对于
对待
对方
对比
将
小
尔
尔后
尔尔
尚且
就
就是
就是了
就是说
就算
就要
尽
尽管
尽管如此
岂但
己
已
已矣
巴
巴巴
年
并
并且
庶乎
庶几
开外
开始
归
归齐
当
当地
当然
当着
彼
彼时
彼此
往
待
很
得
得了
怎
怎么
怎么办
怎么样
怎奈
怎样
总之
总的来看
总的来说
总的说来
总而言之
恰恰相反
您
惟其
慢说
我
我们
或
或则
或是
或曰
或者
截至
所
所以
所在
所幸
所有
才
才能
打
打从
把
抑或
拿
按
按照
换句话说
换言之
据
据此
接着
故
故此
故而
旁人
无
无宁
无论
既
既往
既是
既然
日
时
时候
是
是以
是的
更
曾
替
替代
最
月
有
有些
有关
有及
有时
有的
望
朝
朝着
本
本人
本地
本着
本身
来
来着
来自
来说
极了
果然
果真
某
某个
某些
某某
根据
欤
正值
正如
正巧
正是
此
此地
此处
此外
此时
此次
此间
毋宁
每
每当
比
比及
比如
比方
没奈何
沿
沿着
漫说
点
焉
然则
然后
然而
照
照着
犹且
犹自
甚且
甚么
甚或
甚而
甚至
甚至于
用
用来
由
由于
由是
由此
由此可见
的
的确
的话
直到
相对而言
省得
看
眨眼
着
着呢
矣
矣乎
矣哉
离
秒
称
竟而
第
等
等到
等等
简言之
管
类如
紧接着
纵
纵令
纵使
纵然
经
经过
结果
给
继之
继后
继而
综上所述
罢了
者
而
而且
而况
而后
而外
而已
而是
而言
能
能否
腾
自
自个儿
自从
自各儿
自后
自家
自己
自打
自身
至
至于
至今
至若
致
般的
若
若夫
若是
若果
若非
莫不然
莫如
莫若
虽
虽则
虽然
虽说
被
要
要不
要不是
要不然
要么
要是
譬喻
譬如
让
许多
论
设使
设或
设若
诚如
诚然
该
说
说来
请
诸
诸位
诸如
谁
谁人
谁料
谁知
贼死
赖以
赶
起
起见
趁
趁着
越是
距
跟
较
较之
边
过
还
还是
还有
还要
这
这一来
这个
这么
这么些
这么样
这么点儿
这些
这会儿
这儿
这就是说
这时
这样
这次
这般
这边
这里
进而
连
连同
逐步
通过
遵循
遵照
那
那个
那么
那么些
那么样
那些
那会儿
那儿
那时
那样
那般
那边
那里
都
鄙人
鉴于
针对
阿
除
除了
除外
除开
除此之外
除非
随
随后
随时
随着
难道说
零
非
非但
非徒
非特
非独
靠
顺
顺着
首先'''

stop_words = stop_words + jap.split('\n') + korean.split('\n') + chinese.split('\n')
len(stop_words)

In [ ]:
ps = PorterStemmer()
corpus = []
for i in df.text:
    new = re.sub('[^a-zA-z]', ' ',i)
    new = new.lower()
    new = nltk.word_tokenize(new)
    new = [ps.stem(i) for i in new if i not in stop_words]
    new = ' '.join(new)
    corpus.append(new)
    
df.text = corpus
df.head()

In [ ]:
train,test = train_test_split(df, train_size=0.7, stratify=df.Sentiment, random_state=100)

In [ ]:
X_train = train.text
y_train = train.Sentiment

X_test = test.text
y_test = test.Sentiment

In [ ]:
# folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# mnb = Pipeline([('cnt',CountVectorizer()),
#                ('mnb',MultinomialNB())])
# cv_score = cross_val_score(mnb, X_train, y_train, cv=folds)
# cv_score.mean()

In [ ]:
# # help(MultinomialNB)
# hyp = [{'cnt__ngram_range':[(1,1),(1,2),(2,1),(2,2)],
#        'mnb__alpha': [0.0005,0.001,0.01,0.1,0.13,0.15,0.16,0.18,0.2,0.4,0.6,0.8,1.0,1.2,1.5,1.7,2]}]

# grid = GridSearchCV(estimator=mnb, param_grid=hyp, cv=folds, n_jobs=-1, scoring='accuracy',
#                    verbose=True, return_train_score=True)
# grid.fit(X_train,y_train)

# grid.best_score_, grid.best_estimator_

## Neural Network

In [ ]:
vec = CountVectorizer(max_features=5000)
X_train_transformed = vec.fit_transform(X_train)
X_test_transformed = vec.transform(X_test)

In [ ]:
X_train = pd.DataFrame(X_train_transformed.toarray(),columns=vec.vocabulary_)
X_test = pd.DataFrame(X_test_transformed.toarray(),columns=vec.vocabulary_)
X_train.shape

In [ ]:
def one_hot(lst):
    q = np.zeros([2,len(lst)])
    for i in enumerate(lst):
        if i[1]==1:
            q[1][i[0]] = 1
        elif i[1]==0:
            q[0][i[0]] = 1
    return q.T

In [ ]:
y_train_trans = one_hot(y_train)
y_test_trans = one_hot(y_test)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [ ]:
model = Sequential()
model.add(Dense(10000, input_dim=5000, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(600, activation='relu'))
model.add(Dense(400, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(loss='BinaryCrossentropy', optimizer='adam', metrics='accuracy')

model.fit(np.array(X_train),y_train_trans, batch_size=25, epochs=10)

In [ ]:
model.evaluate(np.array(X_train), y_train_trans)

In [ ]:
model.evaluate(np.array(X_test), y_test_trans)